# IPARO Implementation using IPFS

This notebook demonstrates a simple implementation of InterPlanetary Archival Record Objects (IPAROs) using IPFS. We'll create, store, and link IPAROs, and explore how to retrieve and navigate between them.

## Prerequisites

1. Install IPFS and ensure it's running on your system with:

   ```bash
   ipfs daemon
2. Install the `requests` library for Python to interact with the IPFS HTTP API.

   ```bash
   pip install requests

In [8]:
import json
import requests
import time
from datetime import datetime, UTC

# IPFS API URL
ipfs_api_url = 'http://127.0.0.1:5001/api/v0'

## Step 2: Define Functions for Creating and Storing IPAROs

We will define functions to create IPAROs, add them to IPFS, and update links between them.

In [2]:
def create_iparo(content, prev_cids=None, next_cid=None):
    """
    Create an IPARO with the given content and links to previous and next IPAROs.

    Args:
        content (str): The content of the IPARO.
        prev_cids (list): List of CIDs of the previous IPAROs.
        next_cid (str): CID of the next IPARO.

    Returns:
        dict: The created IPARO.
    """
    iparo = {
        'content': content,
        'prev_cids': prev_cids or [],
        'next_cid': next_cid,
        'timestamp': datetime.now(UTC).isoformat()
    }
    return iparo


def add_to_ipfs(iparo):
    """
    Add the given IPARO to IPFS and return its CID.

    Args:
        iparo (dict): The IPARO to add to IPFS.

    Returns:
        str: The CID of the added IPARO.
    """
    iparo_json = json.dumps(iparo)
    response = requests.post(f'{ipfs_api_url}/add', files={'file': iparo_json})
    cid = response.json()['Hash']
    return cid

## Step 3: Create Functions for Different Linkages

### 3.1: Create a Chain with Each Node Linking to All Preceding Nodes

In [3]:
def create_chain_all_preceding(num_nodes):
    """
    Create a chain of IPAROs where each node links to all preceding nodes.

    Args:
        num_nodes (int): The number of nodes to create.

    Returns:
        list: A list of CIDs for the created IPAROs.
    """
    cids = []
    for i in range(num_nodes):
        content = f"Node {i + 1}"
        prev_cids = cids.copy() if i > 0 else []
        iparo = create_iparo(content, prev_cids=prev_cids)
        cid = add_to_ipfs(iparo)
        cids.append(cid)
    return cids


# Example usage
cids_all_preceding = create_chain_all_preceding(5)
print("CIDs (All Preceding):", cids_all_preceding)

CIDs (All Preceding): ['QmdTs3VM5ov6JGknoeu9nytH729oRLRE29Pg2x9G4HZy1D', 'QmP9Zmco1uLMNAq5iR55DGdZtdpDS4iobiRQu4BW3qcgMp', 'QmPJCRMSrBARZJToBvvigFbJF5Ng6jGtz8JLeGjnNgM5ts', 'QmQ4mNy9daaYzqM12re8kPBEzvLeJoxhiErDVycUShtgiJ', 'QmfV3tNijju3WWpV2m59NqVbRxnbZ74c1oKjq2meAczHbR']


### 3.2: Create a Chain with Each Node Linking Only to the Prior Node


In [4]:
def create_chain_prior_node(num_nodes):
    """
    Create a chain of IPAROs where each node links only to the prior node.

    Args:
        num_nodes (int): The number of nodes to create.

    Returns:
        list: A list of CIDs for the created IPAROs.
    """
    cids = []
    prev_cid = None
    for i in range(num_nodes):
        content = f"Node {i + 1}"
        prev_cids = [prev_cid] if prev_cid else []
        iparo = create_iparo(content, prev_cids=prev_cids)
        cid = add_to_ipfs(iparo)
        cids.append(cid)
        prev_cid = cid
    return cids


# Example usage
cids_prior_node = create_chain_prior_node(5)
print("CIDs (Prior Node):", cids_prior_node)

CIDs (Prior Node): ['QmVVQeyeZ3g9RJSa5nWC239grSd2bezp3CHenyELNCwzKx', 'QmSCq4EDaF95RiRcb9K3D7i7SrHtjFS5scKwCq1goUbhzb', 'QmNYgraeXmYaLqNFvjNxCdm4AFPNf7RLMJ74RYG6bw9a7e', 'QmdM1P1s5QggZyZh8qwdFnr8SwmWkuoKVpqk2bEqHD5g8y', 'QmPRyEVoeojvHaLqTH9Wgv5CLhhmvj3Li2PzAcsc6PNpu3']


In [6]:
def get_iparo(cid):
    """
    Retrieve an IPARO from IPFS using its CID.

    Args:
        cid (str): The CID of the IPARO to retrieve.

    Returns:
        dict: The retrieved IPARO.
    """
    response = requests.post(f'{ipfs_api_url}/cat?arg={cid}')
    iparo_json = response.content.decode('utf-8')
    iparo = json.loads(iparo_json)
    return iparo


# Retrieve and print IPARO 1
retrieved_iparo1 = get_iparo(cids_prior_node[0])
print("Retrieved IPARO 1:", json.dumps(retrieved_iparo1, indent=2), "\n")

# Retrieve and print IPARO 2
retrieved_iparo2 = get_iparo(retrieved_iparo1['next_cid'])
print("Retrieved IPARO 2:", json.dumps(retrieved_iparo2, indent=2))

Retrieved IPARO 1: {
  "content": "Node 1",
  "prev_cids": [],
  "next_cid": null,
  "timestamp": "2024-07-25T20:29:51.948928+00:00"
} 

Retrieved IPARO 2: {
  "Message": "invalid path \"None\": path does not have enough components",
  "Code": 0,
  "Type": "error"
}


## Step 4: Metric tests

### 4.1: Storage Efficiency


In [7]:
def measure_storage_efficiency(iparo):
    """
    Measure the size of an IPARO.

    Args:
        iparo (dict): The IPARO to measure.

    Returns:
        int: The size of the IPARO in bytes.
    """
    iparo_json = json.dumps(iparo)
    return len(iparo_json)


def test_storage_efficiency():
    content = "This is a test IPARO"
    iparo = create_iparo(content)
    original_size = len(content)
    iparo_size = measure_storage_efficiency(iparo)
    print(f"Original content size: {original_size} bytes")
    print(f"IPARO size: {iparo_size} bytes")


test_storage_efficiency()

Original content size: 20 bytes
IPARO size: 119 bytes


### 4.2: Retrieval Time

In [9]:
def measure_retrieval_time(cid):
    """
    Measure the retrieval time of an IPARO from IPFS.

    Args:
        cid (str): The CID of the IPARO to retrieve.

    Returns:
        float: The retrieval time in seconds.
    """
    start_time = time.time()
    iparo = get_iparo(cid)
    end_time = time.time()
    return end_time - start_time


def test_retrieval_time(cids):
    times = [measure_retrieval_time(cid) for cid in cids]
    avg_time = sum(times) / len(times)
    print(f"Average retrieval time: {avg_time:.4f} seconds")


test_retrieval_time(cids_prior_node)

Average retrieval time: 0.0025 seconds


### 4.3: Version Control

In [10]:
def test_version_control():
    content_v1 = "This is version 1"
    iparo_v1 = create_iparo(content_v1)
    cid_v1 = add_to_ipfs(iparo_v1)

    content_v2 = "This is version 2"
    iparo_v2 = create_iparo(content_v2, prev_cids=[cid_v1])
    cid_v2 = add_to_ipfs(iparo_v2)

    retrieved_iparo_v2 = get_iparo(cid_v2)
    assert retrieved_iparo_v2['prev_cids'] == [
        cid_v1], "Version control test failed"
    print("Version control test passed")


test_version_control()

Version control test passed


### 4.4: Scalability


In [11]:
def test_scalability(num_nodes):
    cids = create_chain_prior_node(num_nodes)
    test_retrieval_time(cids)


test_scalability(100)

Average retrieval time: 0.0020 seconds


### 4.5: Data Integrity


In [12]:
def test_data_integrity(cid, original_content):
    retrieved_iparo = get_iparo(cid)
    assert retrieved_iparo['content'] == original_content, "Data integrity test failed"
    print("Data integrity test passed")


test_data_integrity(cids_prior_node[0], "Node 1")

Data integrity test passed


### 4.6: Redundancy and Reliability


In [ ]:
def test_redundancy(cid):
    # To test redundancy, we would need to run IPFS in a multi-node setup
    # and simulate node failures. This is a placeholder function.
    print("Redundancy test requires a multi-node setup and is not implemented here")


test_redundancy(cids_prior_node[0])

### 4.7: Access Control

In [ ]:
def test_access_control():
    # IPFS does not natively support access control. This would require additional layers.
    # This is a placeholder function.
    print("Access control test is not implemented as IPFS does not natively support it")


test_access_control()

## Summary

This notebook demonstrates a foundational implementation of InterPlanetary Archival Record Objects (IPAROs) using IPFS. We created, stored, linked, and retrieved IPAROs, providing a basic understanding of how to work with decentralized web archiving objects.

### Implemented Features:

1. **IPARO Creation**: Defined a function to create IPAROs with content, links to previous and next IPAROs, and a timestamp.
2. **Storing IPAROs in IPFS**: Developed a function to add IPAROs to IPFS and retrieve their unique content identifiers (CIDs).
3. **Linkage Methods**:
   - **All Preceding Nodes**: Implemented a function to create a chain where each IPARO links to all preceding IPAROs.
   - **Prior Node Only**: Implemented a function to create a chain where each IPARO links only to the immediately preceding IPARO.
4. **Retrieving and Navigating IPAROs**: Demonstrated how to retrieve IPAROs from IPFS and navigate between linked IPAROs.
5. **Metric Tests**:
   - **Storage Efficiency**: Measured the size of IPAROs and compared it with the original content size.
   - **Retrieval Time**: Measured the time taken to retrieve IPAROs from IPFS.
   - **Version Control**: Tested the linking and retrieval of multiple versions of content.
   - **Scalability**: Evaluated the performance with a large number of IPAROs.
   - **Data Integrity**: Verified the integrity of the stored and retrieved content.
   - **Redundancy and Reliability**: Placeholder for testing in a multi-node setup.
   - **Access Control**: Placeholder as IPFS does not natively support it.
